In [2]:
!pip install qiskit-nature[pyscf] -U

## FROM QISKIT TUTORIAL: https://qiskit.org/ecosystem/nature/tutorials/03_ground_state_solvers.html

  Obtaining dependency information for qiskit-nature[pyscf] from https://files.pythonhosted.org/packages/9c/91/b2867b7cc8b0e81940d2a04e41f04024467e8055cf964ea4e22213f7e9a1/qiskit_nature-0.6.2-py3-none-any.whl.metadata
  Obtaining dependency information for qiskit-terra>=0.24 from https://files.pythonhosted.org/packages/4b/ef/3b735b3a8f17085dc8f31e3c449744ee025a1a594ba835d9f1db2d1a6ebe/qiskit_terra-0.25.1-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.4 from https://files.pythonhosted.org/packages/a3/d3/f88285098505c8e5d141678a24bb9620d902c683f11edc1eb9532b02624e/scipy-1.11.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 1.8 MB/s eta 0:00:00
  Obtaining dependency information for scikit-learn>=0.20.0 from https://files.pythonhosted.org/packages/d4/61/966d3238f6cbcbb13350d31bd0accfc5efdf9e349cd2a42d9761b8b67a18/scikit_learn-1.3.0-cp39-cp39-many

In [3]:
!pip show qiskit_nature 
!pip show pyscf

#Soln found here: https://quantumcomputing.stackexchange.com/questions/26927/help-on-pyscf-library

Name: qiskit-nature
Version: 0.6.2
Summary: Qiskit Nature: A library of quantum computing chemistry/physics experiments
Home-page: https://github.com/Qiskit/qiskit-nature
Author: Qiskit Nature Development Team
Author-email: hello@qiskit.org
License: Apache-2.0
Location: /opt/conda/lib/python3.9/site-packages
Requires: h5py, numpy, psutil, qiskit-terra, rustworkx, scikit-learn, scipy, setuptools, typing-extensions
Required-by: 
Name: pyscf
Version: 2.3.0
Summary: PySCF: Python-based Simulations of Chemistry Framework
Home-page: http://www.pyscf.org
Author: Qiming Sun
Author-email: osirpt.sun@gmail.com
License: Apache License 2.0
Location: /opt/conda/lib/python3.9/site-packages
Requires: h5py, numpy, scipy
Required-by: 


In [4]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit_nature.second_q.mappers import JordanWignerMapper

from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit.circuit.library import TwoLocal

In [5]:
# Creating PySCF driver for respective molecule based on xyz file format 
# Specifically for Nitrogen (N2) molecule:
driver = PySCFDriver(
    atom="H 0 0 -0.37; H 0 0 0.37", ## H2 <Just Right>; charge/spin:0/0
    #atom="N 0 0 0; N 1.098 0 0", ##N2 <TOO LONG>; charge/spin:0/0
    #atom="N 0 0 -0.069; H 0 0.943 0.321; H -0.817 -0.472 0.321; H 0.817 -0.472 0.321", ##NH3 <TOO LONG>; charge/spin:0/0
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Creating the Electronic Structure (es) problem
es_problem = driver.run()

/opt/conda/lib/python3.9/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [6]:
# Transforms the qubits (from initial '0' state to representing H_el in 2nd quantization form)
mapper = JordanWignerMapper()

In [7]:
# Classical routine
numpy_solver = NumPyMinimumEigensolver()

In [8]:
# Ansatz: Parameterized Quantum Circuit (Theta)
# Typically, we use UCCSD (unitary coupled cluster), which is chemistry-inspired
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(  #HF provided cheap yet good starting approx. value 
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

# Estimator = Qiskit's primitive machine categorization; more suitable for VQE 
# other types is Sampler (which is used in other scenarios)
vqe_solver = VQE(Estimator(), ansatz, SLSQP())

# Initial parameter values 
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

/opt/conda/lib/python3.9/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


In [9]:
# Custom TwoLocal parameterized quantum circuit using:
    # Hadamard and RX gates for rotation/superposition
    # Controlled-Z gates for 'full' entanglement
    # 2 repetitions/layers
    
tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=3,
    parameter_prefix="y",
)

# Second VQE (this time not using UCCSD)
vqe_solver2 = VQE(Estimator(), tl_circuit, SLSQP())

#### VQE vs. VQE2 vs. Classical-Solver

In [10]:
print("-----FROM VQE 1-----\n")

# Using VQE and JW-mapper in classical routine (i.e., GroundStateSolver) to find ground state energy of molecule
calc = GroundStateEigensolver(mapper, vqe_solver)

result_vqe1 = calc.solve(es_problem)
print(result_vqe1)

-----FROM VQE 1-----

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.852388173512
  - computed part:      -1.852388173512
~ Nuclear repulsion energy (Hartree): 0.715104339081
> Total ground state energy (Hartree): -1.137283834431
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  -0.000000216518]
    - computed part:      [0.0  0.0  -0.000000216518]
  > Dipole moment (a.u.): [0.0  0.0  0.000000216518]  Total: 0.000000216518
                 (debye): [0.0  0.0  0.000000550335]  Total: 0.000000550335
 


In [11]:
print("-----FROM VQE 2-----\n")

# Using VQE and JW-mapper in classical routine (i.e., GroundStateSolver) to find ground state energy of molecule
calc = GroundStateEigensolver(mapper, vqe_solver2)

result_vqe2 = calc.solve(es_problem)
print(result_vqe2)

-----FROM VQE 2-----

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.852388098092
  - computed part:      -1.852388098092
~ Nuclear repulsion energy (Hartree): 0.715104339081
> Total ground state energy (Hartree): -1.137283759011
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  -0.000000008646]
    - computed part:      [0.0  0.0  -0.000000008646]
  > Dipole moment (a.u.): [0.0  0.0  0.000000008646]  Total: 0.000000008646
                 (debye): [0.0  0.0  0.000000021977]  Total: 0.000000021977
 


In [12]:
print("-----CLASSICAL ONLY-----\n")

# GroundStateEigensolver taking only classical algorithm to find GSE
calc = GroundStateEigensolver(mapper, numpy_solver)
result_cls = calc.solve(es_problem)
print(result_cls)

-----CLASSICAL ONLY-----

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.85238817357
  - computed part:      -1.85238817357
~ Nuclear repulsion energy (Hartree): 0.715104339081
> Total ground state energy (Hartree): -1.137283834489
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 
